In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Listing current data on our folder.
import os
print(os.listdir("."))

['Midi notes extraction.ipynb', 'MIDIs', 'query', 'Referensi.txt']


In [17]:
# Defining some constants and creating a new folder for MIDIs.
midi_path = "MIDIs"
sonic_folder = "sonic"

# Some helper methods.    
def concat_path(path, child):
    return path + "/" + child
    
print(os.listdir(midi_path))

['00001.mid']


In [18]:
from music21 import converter, corpus, instrument, midi, note, chord, pitch

def open_midi(midi_path, remove_drums):
    # There is an one-line method to read MIDIs
    # but to remove the drums we need to manipulate some
    # low level MIDI events.
    mf = midi.MidiFile()
    mf.open(midi_path)
    mf.read()
    mf.close()
    if (remove_drums):
        for i in range(len(mf.tracks)):
            mf.tracks[i].events = [ev for ev in mf.tracks[i].events if ev.channel != 10]          

    return midi.translate.midiFileToStream(mf)
    
base_midi = open_midi(concat_path(midi_path, "00001.mid"), False)
base_midi

<music21.stream.Score 0x2388e0e2b48>

In [19]:
def list_instruments(midi):
    partStream = midi.parts.stream()
    print("List of instruments found on MIDI file:")
    for p in partStream:
        aux = p
        print (p.partName)

list_instruments(base_midi)

List of instruments found on MIDI file:
Piano


In [43]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

def try_to_compare(midi_notes):
    import pandas as pd

    print('Midi notes: ',midi_notes)
    # read semitone data from query
    df_query = pd.read_csv('query/person00001-00001-semitone-75-600.csv')
    query_notes = df_query['semitone']

    compute_distance(query_notes, midi_notes)

def compute_distance(ref,hyp):
    import edit_distance

    sm = edit_distance.SequenceMatcher(a=ref, b=hyp)
    print(sm.get_opcodes())
    print(sm.ratio())
    print(sm.get_matching_blocks())

def extract_notes(midi_part):
    parent_element = []
    ret = []
    for nt in midi_part.flat.notes:        
        if isinstance(nt, note.Note):
            ret.append(max(0.0, nt.pitch.ps))
            parent_element.append(nt)
        elif isinstance(nt, chord.Chord):
            for pitch in nt.pitches:
                ret.append(max(0.0, pitch.ps))
                parent_element.append(nt)

    try_to_compare(ret)
    return ret, parent_element

def print_parts_countour(midi):
    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_subplot(1, 1, 1)
    minPitch = pitch.Pitch('C10').ps
    maxPitch = 0
    xMax = 0
    
    # Drawing notes.
    for i in range(len(midi.parts)):
        top = midi.parts[i].flat.notes                  
        y, parent_element = extract_notes(top)
        if (len(y) < 1): continue
            
        x = [n.offset for n in parent_element]
        ax.scatter(x, y, alpha=0.6, s=7)
        
        aux = min(y)
        if (aux < minPitch): minPitch = aux
            
        aux = max(y)
        if (aux > maxPitch): maxPitch = aux
            
        aux = max(x)
        if (aux > xMax): xMax = aux
    
    for i in range(1, 10):
        linePitch = pitch.Pitch('C{0}'.format(i)).ps
        if (linePitch > minPitch and linePitch < maxPitch):
            ax.add_line(mlines.Line2D([0, xMax], [linePitch, linePitch], color='red', alpha=0.1))        

    plt.ylabel("Note index (each octave has 12 notes)")
    plt.xlabel("Number of quarter notes (beats)")
    plt.title('Voices motion approximation, each color is a different instrument, red lines show each octave')
    plt.show()

# Focusing only on 6 first measures to make it easier to understand.
print_parts_countour(base_midi.measures(0, 6))

Midi notes:  [60.0, 62.0, 64.0, 64.0, 62.0, 60.0, 62.0, 64.0, 60.0, 55.0, 60.0, 62.0, 64.0, 64.0, 62.0, 60.0, 62.0, 64.0, 60.0, 60.0, 60.0, 60.0, 64.0, 67.0, 67.0, 67.0, 67.0, 69.0, 67.0, 65.0, 64.0, 62.0, 60.0]
[['delete', 0, 1, 0, 0], ['delete', 1, 2, 0, 0], ['delete', 2, 3, 0, 0], ['delete', 3, 4, 0, 0], ['delete', 4, 5, 0, 0], ['delete', 5, 6, 0, 0], ['delete', 6, 7, 0, 0], ['delete', 7, 8, 0, 0], ['delete', 8, 9, 0, 0], ['delete', 9, 10, 0, 0], ['delete', 10, 11, 0, 0], ['delete', 11, 12, 0, 0], ['delete', 12, 13, 0, 0], ['delete', 13, 14, 0, 0], ['delete', 14, 15, 0, 0], ['delete', 15, 16, 0, 0], ['delete', 16, 17, 0, 0], ['delete', 17, 18, 0, 0], ['delete', 18, 19, 0, 0], ['delete', 19, 20, 0, 0], ['delete', 20, 21, 0, 0], ['delete', 21, 22, 0, 0], ['delete', 22, 23, 0, 0], ['delete', 23, 24, 0, 0], ['delete', 24, 25, 0, 0], ['delete', 25, 26, 0, 0], ['delete', 26, 27, 0, 0], ['delete', 27, 28, 0, 0], ['delete', 28, 29, 0, 0], ['delete', 29, 30, 0, 0], ['delete', 30, 31, 0, 0], 

ValueError: ('Lengths must match to compare', (639,), (33,))